# Move and trim files recorded on Feb 16th, Performance

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [1]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [3]:
%reload_ext autoreload

In [4]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [5]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VA306,5022126,230216,0,DATA-VA306-5022126-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DATA,VN202,5022051,230216,0,DATA-VN202-5022051-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DATA,VN201,5022050,230216,0,DATA-VN201-5022050-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DATA,LS410,5022154,230217,101,DATA-LS410-5022154-23021700_0.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DATA,PC701,5022238,230216,0,DATA-PC701-5022238-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,DATA,PC704,5022244,230217,0,DATA-PC704-5022244-23021700.CSV,csv,95833,2023-02-17 07:22:37+00:00,2023-02-17 12:11:52+00:00,...,[PC704],5022244.0,PC704,NaN,69.0,98.0,0.0,0.0,Stationary,Upright
112,DATA,PC705,5022245,230217,0,DATA-PC705-5022245-23021700.CSV,csv,95659,2023-02-17 07:22:41+00:00,2023-02-17 12:11:41+00:00,...,[PC705],5022245.0,PC705,NaN,80.0,98.0,0.0,0.0,Stationary,Upright
113,DATA,PC703,5022241,230217,0,DATA-PC703-5022241-23021700.CSV,csv,92119,2023-02-17 07:22:42+00:00,2023-02-17 12:03:12+00:00,...,[PC703],5022241.0,PC703,NaN,0.0,100.0,1.0,1.0,Stationary,Prone
114,DATA,PC701,5022238,230217,0,DATA-PC701-5022238-23021700.CSV,csv,93010,2023-02-17 07:22:43+00:00,2023-02-17 12:03:13+00:00,...,[PC701],5022238.0,PC701,NaN,81.0,98.0,0.0,0.0,Stationary,Upright


In [6]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 116
[5022126 5022051 5022050 5022154 5022238 5022009 5022240 5022006 5022241
 5022002 5022244 5022245 3420755 5022256 3420456 5022100 3420427 5022101
 3420426 5022104 3420415 5022110 5022131 5022117 5022057 5022151 5022149
 5022179 5022166 5022184 5022164 5022193 5022163 5022198 5022161 5022202
 5022156 5022203 5022098 5022127 5022204 5022138 5022077 5022139 5022076
 5022141 5022073 5022142 5022062 5022143 5022060 5023926 5022058 5022173
 5022285 5022155]
['VA306' 'VN202' 'VN201' 'LS410' 'PC701' 'VN108' 'PC702' 'VN107' 'PC703'
 'VN106' 'PC704' 'PC705' 'VN105' 'PC707' 'VN104' 'VA301' 'VN103' 'VA302'
 'VN102' 'VA303' 'VN101' 'VA304' 'VA308' 'VA305' 'VN203' 'LS408' 'LS407'
 'BR601' 'WW506' 'BR602' 'WW505' 'BR603' 'WW504' 'BR604' 'WW503' 'BR605'
 'WW502' 'BR606' 'VN210' 'VA307' 'BR607' 'LS401' 'VN209' 'LS402' 'VN208'
 'LS403' 'VN207' 'LS404' 'VN206' 'LS405' 'VN205' 'LS406' 'VN204' 'WW507'
 'Bluetooth02' 'WW501']
[230216 230217]


# discard tiny


In [7]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230216
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [8]:
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VA306,5022126,230216,0,DATA-VA306-5022126-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DATA,VN202,5022051,230216,0,DATA-VN202-5022051-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DATA,VN201,5022050,230216,0,DATA-VN201-5022050-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DATA,PC701,5022238,230216,0,DATA-PC701-5022238-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DATA,VN108,5022009,230216,0,DATA-VN108-5022009-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DATA,PC702,5022240,230216,0,DATA-PC702-5022240-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DATA,VN107,5022006,230216,0,DATA-VN107-5022006-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DATA,PC703,5022241,230216,0,DATA-PC703-5022241-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,DATA,PC704,5022244,230216,0,DATA-PC704-5022244-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,DATA,VN106,5022002,230216,0,DATA-VN106-5022002-23021600.CSV,csv,134,2020-02-02 02:02:00+00:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Define set of recordings to move

We sort files by day and time recording

In [9]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230216
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


Additionally select according to start and stop recording times.

In [11]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-16 9:34:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [12]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-16 07:00:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']> t,:] 

In [13]:
# recordings ended after a certain time
t = pd.to_datetime('2023-02-16 10:30:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [14]:
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
58,DATA,Bluetooth02,5022285,230216,0,DATA-Bluetooth02-5022285-23021600.CSV,csv,510330,2023-02-16 07:10:40+00:00,2023-02-17 07:43:10+00:00,...,[Bluetooth02],5022285.0,Bluetooth02,NaN,26.0,100.0,1.0,1.0,Stationary,Side
59,DATA,LS410,5022154,230216,0,DATA-LS410-5022154-23021600.CSV,csv,567466,2023-02-16 07:10:44+00:00,2023-02-17 12:13:44+00:00,...,[LS410],5022154.0,LS410,NaN,2.0,99.0,0.0,1.0,Stationary,Prone


In [23]:
# show subset of files
s_files.iloc[1,14]

'C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\Extracted SEM Data\\LS410\\DATA-LS410-5022154-23021600.CSV'

In [15]:
s_files.max()

Signal                                                            DATA
DevName                                                          WW508
ID                                                             5023926
Date                                                            230215
Session                                                            102
FileName                               DATA-WW508-5022176-23021501.CSV
FileType                                                           csv
FileSize                                                        203597
RecStart                                     2023-02-15 10:13:51+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 16:50:18+00:00
Duration                                                       37230.0
BatteryStart                                                    4170.0
BatteryEnd                                                      4158.0
Batter

In [16]:
s_files.min()

Signal                                                            DATA
DevName                                                          BR601
ID                                                             3420415
Date                                                            230215
Session                                                              0
FileName                               DATA-BR601-5022179-23021500.CSV
FileType                                                           csv
FileSize                                                           217
RecStart                                     2023-02-15 06:27:44+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 07:27:41+00:00
Duration                                                           0.0
BatteryStart                                                    4082.0
BatteryEnd                                                      3795.0
Batter

These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [15]:
projectsFolder =  'D:\\' # where qiosk files are moved to after export
print(projectsFolder) # where the qiosk files are initially saved by the Equivital Qiosk program

D:\


In [16]:
# set project folder name
projectName = '230216_Lydo_Concerts'
projectPath = projectsFolder + projectName + '\\'

In [17]:
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [18]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

# Set Interval to keep in CSV

We will cut the the CSV files to save on space and presever privacy. 
In this case, we are only taking the 

In [24]:
t1 = pd.to_datetime('2023-02-16 09:15:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]
t2 = pd.to_datetime('2023-02-16 12:30:00.00+0100')
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

[Timestamp('2023-02-16 09:15:00+0100', tz='pytz.FixedOffset(60)'),
 Timestamp('2023-02-16 12:30:00+0100', tz='pytz.FixedOffset(60)')]

In [25]:
(t2-t1).total_seconds()

11700.0

In [31]:
# test cutting
tic = time.time()

dataLoc = s_files.iloc[1,14]
devName = s_files.iloc[1,1]
matched = qex.matched_files(dataLoc,path,seperator) # outputs locations of csv 
for fi in matched:
    if not fi.lower().endswith('.sem'):
        fp = fi.split(seperator)
        fn = fp[-1].split('.')
        fn_new = fn[0]+'-trimmed.csv'
        print(fn_new)
        V = qex.cut_by_time(fi,t1,t2)
        print(V.columns)
        V.to_csv(projectPath + 'CSV\\' + fn_new)
        print(time.time()-tic)
    if fi.lower().endswith('sem'):
        if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
            os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
        out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
        shutil.copy2(fi_loc,out_f)
        print(time.time()-tic)

DATA-LS410-5022154-23021600-trimmed.csv
Index(['SENSOR ID', 'SUBJECT ID', 'SUBJECT AGE', 'HR(BPM)', 'HRC(%)',
       'BELT OFF', 'LEAD OFF', 'MOTION', 'BODY POSITION', 'BATTERY(mV)',
       'TEMPERATURE', 'BUTTON PRESS'],
      dtype='object')
0.08886528015136719
ECG-LS410-5022154-23021600-trimmed.csv
Index(['Lead 1', 'Lead 2'], dtype='object')
85.0978741645813
FASTACC-LS410-5022154-23021600-trimmed.csv
Index(['Vert Accelerometer', 'Lat Accelerometer', 'Long Accelerometer'], dtype='object')
163.4186520576477
RESPACC-LS410-5022154-23021600-trimmed.csv
Index(['Breathing'], dtype='object')
171.3937885761261
RR-LS410-5022154-23021600-trimmed.csv
Index(['Interbeat Interval (MS)'], dtype='object')
171.66338992118835


NameError: name 'fileName' is not defined

# Full Cut Save Move Discard

In [32]:
tac = time.time()

for i,row in s_files.iterrows():
    tic = time.time()
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator)  # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                fn = fileName.split('.')
                fn_new = fn[0]+'-cut.csv'
                out_f = projectPath + 'CSV\\' + fn_new
                print(out_f)
                V = qex.cut_by_time(fi,t1,t2)
                V.to_csv(out_f) # write the trimmed csv to the external harddisk
                
                dout_f = discardPath + 'CSV\\' + fileName
                shutil.move(fi,dout_f) # move the exported csv files to the discard folder
                print(time.time()-tic)
        if fileName.lower().endswith('sem'):
            devName = row['DevName']
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)
            print(time.time()-tic)
            
    print([time.time()-tic, time.time()-tac])

dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230216_Lydo_Concerts\CSV\DATA-WW503-5022161-23021600-cut.csv
0.3651895523071289
D:\230216_Lydo_Concerts\CSV\ECG-WW503-5022161-23021600-cut.csv
22.74461054801941
D:\230216_Lydo_Concerts\CSV\FASTACC-WW503-5022161-23021600-cut.csv
47.57980251312256
D:\230216_Lydo_Concerts\CSV\RESPACC-WW503-5022161-23021600-cut.csv
49.7533392906189
D:\230216_Lydo_Concerts\CSV\RR-WW503-5022161-23021600-cut.csv
49.988511085510254
51.00357532501221
[51.00357532501221, 51.00357532501221]
D:\230216_Lydo_Concerts\CSV\DATA-LS404-5022142-23021600-cut.csv
0.0368809700012207
D:\230216_Lydo_Concerts\CSV\ECG-LS404-5022142-23021600-cut.csv
36.274452209472656
D:\230216_Lydo_Concerts\CSV\FASTACC-LS404-5022142-23021600-cut.csv
59.91215944290161
D:\230216_Lydo_Concerts\CSV\RESPACC-LS404-5022142-23021600-cut.csv
63.1763813495636
D:\230216_Lydo_Concerts\CSV\RR-LS404-5022142-23021600-cut.csv
63.38073205947876
64.2818295955658
[64.28283309936523, 115.28640842437744]
D:\230216_Lydo_Concerts\CSV\DATA-VA304-5022110-23021600-cu

70.09391784667969
D:\230216_Lydo_Concerts\CSV\RESPACC-VA306-5022126-23021600-cut.csv
73.50722861289978
D:\230216_Lydo_Concerts\CSV\RR-VA306-5022126-23021600-cut.csv
73.70744967460632
74.56951093673706
[74.56951093673706, 1123.2398657798767]
D:\230216_Lydo_Concerts\CSV\DATA-LS407-5022149-23021600-cut.csv
0.03288984298706055
D:\230216_Lydo_Concerts\CSV\ECG-LS407-5022149-23021600-cut.csv
34.819037199020386
D:\230216_Lydo_Concerts\CSV\FASTACC-LS407-5022149-23021600-cut.csv
66.29759979248047
D:\230216_Lydo_Concerts\CSV\RESPACC-LS407-5022149-23021600-cut.csv
69.2168698310852
D:\230216_Lydo_Concerts\CSV\RR-LS407-5022149-23021600-cut.csv
69.45706701278687
70.41397953033447
[70.41397953033447, 1193.6548418998718]
D:\230216_Lydo_Concerts\CSV\DATA-VN108-5022009-23021600-cut.csv
0.05083012580871582
D:\230216_Lydo_Concerts\CSV\ECG-VN108-5022009-23021600-cut.csv
38.549277544021606
D:\230216_Lydo_Concerts\CSV\FASTACC-VN108-5022009-23021600-cut.csv
64.38209462165833
D:\230216_Lydo_Concerts\CSV\RESPACC

58.952089071273804
[58.95308589935303, 2223.364889860153]
D:\230216_Lydo_Concerts\CSV\DATA-VN206-5022062-23021600-cut.csv
0.0378727912902832
D:\230216_Lydo_Concerts\CSV\ECG-VN206-5022062-23021600-cut.csv
32.99208164215088
D:\230216_Lydo_Concerts\CSV\FASTACC-VN206-5022062-23021600-cut.csv
58.58996534347534
D:\230216_Lydo_Concerts\CSV\RESPACC-VN206-5022062-23021600-cut.csv
61.88442587852478
D:\230216_Lydo_Concerts\CSV\RR-VN206-5022062-23021600-cut.csv
62.08579635620117
63.003726959228516
[63.00472354888916, 2286.3696134090424]
D:\230216_Lydo_Concerts\CSV\DATA-VN203-5022057-23021600-cut.csv
0.030896663665771484
D:\230216_Lydo_Concerts\CSV\ECG-VN203-5022057-23021600-cut.csv
37.205010175704956
D:\230216_Lydo_Concerts\CSV\FASTACC-VN203-5022057-23021600-cut.csv
62.82039284706116
D:\230216_Lydo_Concerts\CSV\RESPACC-VN203-5022057-23021600-cut.csv
65.27833461761475
D:\230216_Lydo_Concerts\CSV\RR-VN203-5022057-23021600-cut.csv
65.41989374160767
66.33184742927551
[66.33184742927551, 2352.701460838

35.975311040878296
D:\230216_Lydo_Concerts\CSV\FASTACC-VN205-5022060-23021600-cut.csv
62.99869132041931
D:\230216_Lydo_Concerts\CSV\RESPACC-VN205-5022060-23021600-cut.csv
65.82947134971619
D:\230216_Lydo_Concerts\CSV\RR-VN205-5022060-23021600-cut.csv
66.01286053657532
66.93584108352661
[66.93584108352661, 3385.9552986621857]
D:\230216_Lydo_Concerts\CSV\DATA-VN207-5022073-23021600-cut.csv
0.042852163314819336
D:\230216_Lydo_Concerts\CSV\ECG-VN207-5022073-23021600-cut.csv
37.03947949409485
D:\230216_Lydo_Concerts\CSV\FASTACC-VN207-5022073-23021600-cut.csv
62.28470063209534
D:\230216_Lydo_Concerts\CSV\RESPACC-VN207-5022073-23021600-cut.csv
64.93515920639038
D:\230216_Lydo_Concerts\CSV\RR-VN207-5022073-23021600-cut.csv
65.11455607414246
65.95873618125916
[65.9597327709198, 3451.9160284996033]
D:\230216_Lydo_Concerts\
230216_Lydo_Concerts


,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VN205,5022060,230216,0,DATA-VN205-5022060-23021600-cut.csv,csv,64201,2023-02-16 08:15:01+00:00,2023-02-16 11:29:46+00:00,...,[VN205],5022060.0,VN205,NaN,66.0,98.0,0.0,0.0,Stationary,Upright
1,DATA,WW506,5022166,230216,0,DATA-WW506-5022166-23021600-cut.csv,csv,64376,2023-02-16 08:15:02+00:00,2023-02-16 11:29:47+00:00,...,[WW506],5022166.0,WW506,NaN,82.0,98.0,0.0,0.0,Stationary,Upright
2,DATA,VA301,5022100,230216,0,DATA-VA301-5022100-23021600-cut.csv,csv,64353,2023-02-16 08:15:02+00:00,2023-02-16 11:29:47+00:00,...,[VA301],5022100.0,VA301,NaN,83.0,98.0,0.0,0.0,Stationary,Upright
3,DATA,WW501,5022155,230216,0,DATA-WW501-5022155-23021600-cut.csv,csv,64416,2023-02-16 08:15:02+00:00,2023-02-16 11:29:47+00:00,...,[WW501],5022155.0,WW501,NaN,79.0,98.0,0.0,0.0,Stationary,Upright
4,DATA,PC701,5022238,230216,0,DATA-PC701-5022238-23021600-cut.csv,csv,64652,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[PC701],5022238.0,PC701,NaN,78.0,48.0,0.0,0.0,Stationary,Upright
5,DATA,VN209,5022077,230216,0,DATA-VN209-5022077-23021600-cut.csv,csv,64085,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[VN209],5022077.0,VN209,NaN,71.0,98.0,0.0,0.0,Stationary,Upright
6,DATA,LS403,5022141,230216,0,DATA-LS403-5022141-23021600-cut.csv,csv,64358,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[LS403],5022141.0,LS403,NaN,94.0,98.0,0.0,0.0,Stationary,Upright
7,DATA,VN208,5022076,230216,0,DATA-VN208-5022076-23021600-cut.csv,csv,64213,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[VN208],5022076.0,VN208,NaN,79.0,98.0,0.0,0.0,Stationary,Upright
8,DATA,VN210,5022098,230216,0,DATA-VN210-5022098-23021600-cut.csv,csv,64252,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[VN210],5022098.0,VN210,NaN,74.0,99.0,0.0,0.0,Stationary,Upright
9,DATA,BR605,5022202,230216,0,DATA-BR605-5022202-23021600-cut.csv,csv,64151,2023-02-16 08:15:04+00:00,2023-02-16 11:29:49+00:00,...,[BR605],5022202.0,BR605,NaN,74.0,95.0,0.0,0.0,Stationary,Upright


In [3]:
1715/66

29.3

In [33]:
dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230216_Lydo_Concerts\
230216_Lydo_Concerts


,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VN205,5022060,230216,0,DATA-VN205-5022060-23021600-cut.csv,csv,64201,2023-02-16 08:15:01+00:00,2023-02-16 11:29:46+00:00,...,[VN205],5022060.0,VN205,NaN,66.0,98.0,0.0,0.0,Stationary,Upright
1,DATA,WW506,5022166,230216,0,DATA-WW506-5022166-23021600-cut.csv,csv,64376,2023-02-16 08:15:02+00:00,2023-02-16 11:29:47+00:00,...,[WW506],5022166.0,WW506,NaN,82.0,98.0,0.0,0.0,Stationary,Upright
2,DATA,WW501,5022155,230216,0,DATA-WW501-5022155-23021600-cut.csv,csv,64416,2023-02-16 08:15:02+00:00,2023-02-16 11:29:47+00:00,...,[WW501],5022155.0,WW501,NaN,79.0,98.0,0.0,0.0,Stationary,Upright
3,DATA,VA301,5022100,230216,0,DATA-VA301-5022100-23021600-cut.csv,csv,64353,2023-02-16 08:15:02+00:00,2023-02-16 11:29:47+00:00,...,[VA301],5022100.0,VA301,NaN,83.0,98.0,0.0,0.0,Stationary,Upright
4,DATA,VN208,5022076,230216,0,DATA-VN208-5022076-23021600-cut.csv,csv,64213,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[VN208],5022076.0,VN208,NaN,79.0,98.0,0.0,0.0,Stationary,Upright
5,DATA,PC701,5022238,230216,0,DATA-PC701-5022238-23021600-cut.csv,csv,64652,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[PC701],5022238.0,PC701,NaN,78.0,48.0,0.0,0.0,Stationary,Upright
6,DATA,VN209,5022077,230216,0,DATA-VN209-5022077-23021600-cut.csv,csv,64085,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[VN209],5022077.0,VN209,NaN,71.0,98.0,0.0,0.0,Stationary,Upright
7,DATA,LS403,5022141,230216,0,DATA-LS403-5022141-23021600-cut.csv,csv,64358,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[LS403],5022141.0,LS403,NaN,94.0,98.0,0.0,0.0,Stationary,Upright
8,DATA,VN210,5022098,230216,0,DATA-VN210-5022098-23021600-cut.csv,csv,64252,2023-02-16 08:15:03+00:00,2023-02-16 11:29:48+00:00,...,[VN210],5022098.0,VN210,NaN,74.0,99.0,0.0,0.0,Stationary,Upright
9,DATA,BR605,5022202,230216,0,DATA-BR605-5022202-23021600-cut.csv,csv,64151,2023-02-16 08:15:04+00:00,2023-02-16 11:29:49+00:00,...,[BR605],5022202.0,BR605,NaN,74.0,95.0,0.0,0.0,Stationary,Upright


## Troubleshooting

In [69]:
shutil.move(fi,out_f)

'D:\\230214_Equivital\\SEM\\VN101\\23021400.SEM'

In [11]:
dfiles = qex.qiosk_recordings(discardPath, 'ToBeDiscarded',seperator)
print(len(dfiles))

585
